<a href="https://colab.research.google.com/github/mohamedalaaaz/testpytroch/blob/main/physics%20space%20ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from __future__ import annotations
import math
import torch
from dataclasses import dataclass

# ------------------------------
# Config & device
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

torch.set_default_dtype(dtype)  # use float64 for astronomy scales

# Physical constants (SI)
G = torch.tensor(6.67430e-11, device=device, dtype=dtype)  # m^3 kg^-1 s^-2

# Softening length (Plummer-like) to avoid singularities in close passes
EPS = torch.tensor(1e7, device=device, dtype=dtype)  # meters (10,000 km)

# Time settings
DT = 3600.0  # seconds per step (1 hour)
STEPS = 24 * 90  # 90 days of simulation

@dataclass
class SystemState:
    pos: torch.Tensor  # (N, 3)
    vel: torch.Tensor  # (N, 3)


def pairwise_accel(pos: torch.Tensor, masses: torch.Tensor) -> torch.Tensor:
    """Compute gravitational accelerations on each body due to all others.
    pos:    (N,3)
    masses: (N,)
    returns acc: (N,3)
    """
    # Compute pairwise displacement vectors r_ij = r_j - r_i
    r = pos.unsqueeze(1) - pos.unsqueeze(0)  # (N,N,3) r_ij = r_i - r_j
    r = -r  # switch to r_ij = r_j - r_i for intuition

    # Distance with softening to avoid division by zero
    dist2 = (r*r).sum(-1) + EPS**2  # (N,N)
    inv_dist3 = dist2.pow(-1.5)     # (N,N)

    # Force magnitude per pair: G * m_i * m_j / |r|^3 times vector r_ij
    # We compute acceleration on i: a_i = sum_j G*m_j * r_ij/|r|^3
    m_j = masses.unsqueeze(0)  # (1,N)
    a = (G * m_j.unsqueeze(-1) * r * inv_dist3.unsqueeze(-1)).sum(dim=1)  # (N,3)
    return a


def leapfrog_step(state: SystemState, masses: torch.Tensor, dt: float) -> SystemState:
    """Velocity Verlet / Leapfrog step (kick-drift-kick)."""
    a0 = pairwise_accel(state.pos, masses)
    v_half = state.vel + 0.5 * dt * a0
    x_new = state.pos + dt * v_half
    a1 = pairwise_accel(x_new, masses)
    v_new = v_half + 0.5 * dt * a1
    return SystemState(x_new, v_new)


def simulate(initial: SystemState, masses: torch.Tensor, steps: int, dt: float) -> torch.Tensor:
    """Simulate and return positions over time: (steps+1, N, 3)."""
    pos_traj = [initial.pos]
    state = SystemState(initial.pos, initial.vel)
    for _ in range(steps):
        state = leapfrog_step(state, masses, dt)
        pos_traj.append(state.pos)
    return torch.stack(pos_traj, dim=0)


# ----------------------------------------------
# Construct a Sun–Earth–Moon ground-truth system
# ----------------------------------------------
# True masses (kg)
M_sun  = 1.98847e30
M_earth= 5.9722e24
M_moon = 7.34767309e22
m_true = torch.tensor([M_sun, M_earth, M_moon], device=device, dtype=dtype)

# Initial positions (meters)
# Sun at origin, Earth at ~1 AU on +x, Moon offset from Earth
AU = 1.495978707e11
R_em = 384400e3

pos0 = torch.tensor([
    [0.0, 0.0, 0.0],             # Sun
    [AU, 0.0, 0.0],               # Earth
    [AU + R_em, 0.0, 0.0],        # Moon (along x from Earth)
], device=device, dtype=dtype)

# Initial velocities (m/s): Earth ~29.78 km/s in +y; Moon ~1.022 km/s about Earth
v_earth = 29780.0
v_moon_rel = 1022.0

vel0 = torch.tensor([
    [0.0, 0.0, 0.0],
    [0.0, v_earth, 0.0],
    [0.0, v_earth + v_moon_rel, 0.0],
], device=device, dtype=dtype)

initial_state_true = SystemState(pos0, vel0)

with torch.no_grad():
    pos_traj_true = simulate(initial_state_true, m_true, STEPS, DT)  # (T,N,3)

# Create noisy "observations" every k steps (e.g., daily)
OBS_EVERY = 24  # observe once per day (24h)
obs_idx = torch.arange(0, STEPS+1, OBS_EVERY, device=device)
noise_std = 5e5  # 500 km observation noise

obs_pos = pos_traj_true[obs_idx] + noise_std * torch.randn_like(pos_traj_true[obs_idx])

# -------------------------------------------------
# Inverse problem: estimate masses from observations
# -------------------------------------------------
# We assume initial positions/velocities and G are known.
# Parameterize masses with softplus to keep them positive.

m_init_guess = torch.tensor([1.5e30, 3.0e24, 1.0e23], device=device, dtype=dtype)  # poor guesses
raw_params = torch.log(torch.expm1(m_init_guess))  # inverse softplus for initialization
raw_params = torch.nn.Parameter(raw_params)

optimizer = torch.optim.Adam([raw_params], lr=1e1)  # relatively high LR; double precision helps
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20, verbose=False)

# Optional: weight the Earth/Moon observations more than the Sun if desired
obs_weights = torch.tensor([1.0, 5.0, 10.0], device=device, dtype=dtype)  # (N,)


def loss_fn(masses_pos: torch.Tensor) -> torch.Tensor:
    masses = torch.nn.functional.softplus(masses_pos)
    traj = simulate(initial_state_true, masses, STEPS, DT)  # (T,N,3)
    pred_obs = traj[obs_idx]  # align with observation times
    diff = (pred_obs - obs_pos)  # (T_obs,N,3)
    # Weighted MSE across bodies
    w = obs_weights.view(1, -1, 1)
    mse = (w * diff.pow(2)).mean()
    # Add gentle L2 prior to keep masses near astrophysical scale (stabilization)
    prior = 1e-6 * ((masses / m_true - 1.0).pow(2)).mean()
    return mse + prior


# Optimization loop
MAX_ITERS = 400
for it in range(1, MAX_ITERS + 1):
    optimizer.zero_grad()
    L = loss_fn(raw_params)
    L.backward()
    torch.nn.utils.clip_grad_norm_([raw_params], max_norm=1e12)
    optimizer.step()
    scheduler.step(L)

    if it % 50 == 0 or it == 1:
        masses_est = torch.nn.functional.softplus(raw_params).detach()
        rel_err = (masses_est - m_true).abs() / m_true
        print(f"iter {it:4d} | loss={L.item():.3e} | M_est=[{masses_est[0]:.3e}, {masses_est[1]:.3e}, {masses_est[2]:.3e}] | rel_err=[{rel_err[0]:.2e}, {rel_err[1]:.2e}, {rel_err[2]:.2e}]")

# Final report
masses_est = torch.nn.functional.softplus(raw_params).detach()
print("\nTrue masses (kg):   ", m_true.cpu().numpy())
print("Estimated masses (kg):", masses_est.cpu().numpy())
print("Relative errors:     ", ((masses_est - m_true).abs() / m_true).cpu().numpy())

# Optional: simulate with estimated masses for visualization / further analysis
with torch.no_grad():
    pos_traj_est = simulate(initial_state_true, masses_est, STEPS, DT)




/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


iter    1 | loss=nan | M_est=[nan, nan, nan] | rel_err=[nan, nan, nan]
iter   50 | loss=nan | M_est=[nan, nan, nan] | rel_err=[nan, nan, nan]
iter  100 | loss=nan | M_est=[nan, nan, nan] | rel_err=[nan, nan, nan]
iter  150 | loss=nan | M_est=[nan, nan, nan] | rel_err=[nan, nan, nan]
iter  200 | loss=nan | M_est=[nan, nan, nan] | rel_err=[nan, nan, nan]
iter  250 | loss=nan | M_est=[nan, nan, nan] | rel_err=[nan, nan, nan]
iter  300 | loss=nan | M_est=[nan, nan, nan] | rel_err=[nan, nan, nan]
iter  350 | loss=nan | M_est=[nan, nan, nan] | rel_err=[nan, nan, nan]
iter  400 | loss=nan | M_est=[nan, nan, nan] | rel_err=[nan, nan, nan]

True masses (kg):    [1.98847000e+30 5.97220000e+24 7.34767309e+22]
Estimated masses (kg): [nan nan nan]
Relative errors:      [nan nan nan]


'\nExtensions / Ideas\n- Fit initial states (pos/vel) jointly with masses using the same approach.\n- Add solar radiation pressure or J2 oblateness terms for satellites.\n- Switch to canonical units where G=1 to improve conditioning and larger time steps.\n- Replace the explicit simulator with a learned correction using a Graph Neural Network.\n- Use torch.compile() (PyTorch 2+) to speed up the integrator on GPU.\n- Add differentiable event handling (e.g., close approaches) with smooth penalties.\n'